In [ ]:
import os
import pandas as pd
import tensorflow as tf
import cv2

# specify the directory where images are stored
img_dir = r'C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train_image'
# specify the directory of the csv file
csv_file = r'C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train_skin.csv'

In [ ]:
# Define functions to convert different types of data into tf.train.Feature
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def create_tf_example(img_path, annotation):
    img = cv2.imread(img_path)
    img_encoded = cv2.imencode('.jpg', img)[1].tostring()

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/encoded': _bytes_feature(img_encoded),
        'image/object/bbox/xmin': _float_feature(annotation['x']),
        'image/object/bbox/xmax': _float_feature(annotation['x'] + annotation['width']),
        'image/object/bbox/ymin': _float_feature(annotation['y']),
        'image/object/bbox/ymax': _float_feature(annotation['y'] + annotation['height']),
        'image/object/class/label': _int64_feature(annotation['class_id']),
    }))

    return tf_example

In [ ]:
# load csv
df = pd.read_csv(csv_file)

# Get unique class names and create a mapping from class names to integer IDs
unique_classes = df['lesions'].unique()
class_dict = {name: idx for idx, name in enumerate(unique_classes)}

# Path where the diseases.names and TFRecord files will be saved
save_path = r'C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/tfrecod'

# Write the class names to a text file
with open(os.path.join(save_path, 'diseases.names'), 'w') as f:
    for disease in unique_classes:
        f.write(disease + '\n')

# Create TFRecords
with tf.io.TFRecordWriter(os.path.join(save_path, 'train.tfrecord')) as writer:
    for img_id, group in df.groupby('Raw data ID'):
        img_path = os.path.join(img_dir, img_id)
        for _, row in group.iterrows():
            annotation = {
                'x': row['x'] / 1920,
                'y': row['y'] / 1080,
                'width': row['width'] / 1920,
                'height': row['height'] / 1080,
                'class_id': class_dict[row['lesions']]
            }
            tf_example = create_tf_example(img_path, annotation)
            writer.write(tf_example.SerializeToString())

In [ ]:
# Clone the project
!git clone https://github.com/zzh8829/yolov3-tf2.git
%cd yolov3-tf2

# Install requirements
!pip install -r requirements.txt

# Prepare the configuration file
with open('C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/tfrecod/diseases.names', 'w') as f:
    for disease in unique_classes:
        f.write(disease + '\n')

# Convert darknet weights to tensorflow model
!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.tf

# Train model
!python train.py \
  --dataset C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/train.tfrecord \
  --classes C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/diseases.names \
  --num_classes len(unique_classes) \
  --mode fit --transfer darknet

In [ ]:
model.save_weights("yolo_model_weights.h5")

In [ ]:
!python detect.py --weights ./checkpoints/yolov3.tf \
  --classes C:/Users/oceanlightai/Desktop/datasets/pet_skin/train/diseases.names \
  --image C:/Users/oceanlightai/Desktop/datasets/pet_skin/valid/IMG_D_A1_011078.jpg